In [1]:
import cv2
import numpy as np
import os

In [2]:
exam = '25080225_08_2019_08_37_59'

dataset_path = '{}/pupil_deep/eye_test/out/{}/'.format(os.getcwd().replace('/auxiliaries',''), exam)
dataset_out = '{}/pupil_deep/eye_test/out/'.format(os.getcwd().replace('/auxiliaries',''), exam)

In [3]:
dataset_path

'/media/marcos/Dados/Projects/PupilDeep/pupil_deep/eye_test/out/25080225_08_2019_08_37_59/'

In [4]:
original_files = []
image_process = []

for file in os.listdir(dataset_path):
    if ('image_' in file) and ('image_process' not in file):
        original_files.append(file)
    
    if 'image_process' in file:
        image_process.append(file)
        

print('original: {}, examples: {}'.format(len(original_files), original_files[0: 10: 1]))
print('process: {}, examples: {}'.format(len(image_process), image_process[0: 10: 1]))

original: 300, examples: ['image_1.png', 'image_10.png', 'image_100.png', 'image_101.png', 'image_102.png', 'image_103.png', 'image_104.png', 'image_105.png', 'image_106.png', 'image_107.png']
process: 300, examples: ['image_process_40.png', 'image_process_41.png', 'image_process_42.png', 'image_process_43.png', 'image_process_44.png', 'image_process_45.png', 'image_process_46.png', 'image_process_47.png', 'image_process_48.png', 'image_process_49.png']


In [5]:
img_array = []
size = None

if len(original_files) == len(image_process):
    for i in range(1, len(original_files)):
        path_img_orig = 'original_{}.png'.format(i)
        path_img_proc = 'image_process_{}.png'.format(i)
        path_img_final = 'image_{}.png'.format(i)
        
        img_orig = cv2.imread('{}/{}'.format(dataset_path, path_img_orig))
        img_proc = cv2.imread('{}/{}'.format(dataset_path, path_img_proc))
        img_final = cv2.imread('{}/{}'.format(dataset_path, path_img_final))
        
        img_presentation = cv2.hconcat([img_orig, img_proc, img_final])
        img_array.append(img_presentation)
        
        if size is None:
            height, width, _ = img_presentation.shape
            size = (width,height)

In [6]:
out = cv2.VideoWriter('{}/{}.mp4'.format(dataset_out, exam), cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()